<a href="https://colab.research.google.com/github/Ok3ks/CLAS-LONG-/blob/main/CNN_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install sklearn
%pip transformers
%pip install huggingface
%pip install torch 
%pip install pandas
%pip install sklearn|
%pip install datasets
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "transformers"
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: -c: line 1: syntax error: unexpected end of file
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset, load_metric
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,get_scheduler
from torch.optim import AdamW

from spacy import tokenizer
from spacy.lang.en import English
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
accuracy = load_metric('accuracy'); f1_score = load_metric('f1'); perplexity = load_metric('perplexity')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [4]:
def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return f1_score.compute(predictions, references = label)

In [5]:
from transformers import BertModel
#Using BERT Embeddings, i.e input ids

model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
max_input_length = 450

def bert_tokenizer(input):
  return tokenizer(input['text'], truncation=True, padding = True, max_length=max_input_length, return_tensors = 'pt')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
class CNN(nn.Module):

  #Params represents the parameters of a conv1d function with input, output, kernel size and stride.


  def __init__(self):
    self.conv = nn.conv1d(768, 100, 5, stride = 1)
    self.conv = nn.conv1d(100, 1, 3, stride = 1)
    self.pool = nn.MaxPool1d(kernel_size = 5, stride = 1) #Number of words it skips 
    self.relu = nn.ReLU()
    
  def forward(self,input):
    out = self.conv(input)
    out = self.pool(out)
    out = self.relu(out)
    self.Linear = nn.Linear(out.shape[-1], 2)
    out = self.Linear(out)

In [7]:
imdb = load_dataset("imdb")
imdb = imdb.shuffle(seed = 42)

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
#0 corresponds to negative review while 1 corresponds to positive reviews
label2text = {0:'negative', 1: 'positive'}

train_imdb = imdb['train']
test_imdb = imdb['test']
set(train_imdb['label'])

imdb_train_text = {i:x for i,x in enumerate(train_imdb['text'])}
imdb_test_text = {i:x for i,x in enumerate(test_imdb['text'])}

temp = []
imdb_labels_text_train = {i:temp.append(x) for i,x in zip(train_imdb['label'], train_imdb['text'])}

temp = []
imdb_labels_text_test = {i:temp.append(x) for i,x in zip(test_imdb['label'], test_imdb['text'])}

imdb_train_labels = {i:x for i,x in enumerate(train_imdb['label'])}
imdb_test_labels = {i:x for i,x in enumerate(test_imdb['label'])}

In [9]:
imdb_tokenized = bert_tokenizer(train_imdb)

In [69]:
temp_conv = torch.nn.Conv1d(768, 30, 5, stride = 3)
temp_conv_2 = torch.nn.Conv1d(30, 1, 1, stride = 1)

#Using multiple convolutional filters

In [ ]:
import torch 
hidden_states = []; hidden_states_2 = []

with torch.no_grad():
  output = model(imdb_tokenized['input_ids'][:500],imdb_tokenized['token_type_ids'][:500])
  hidden_states.append(output[2][-2]) #Using output representation of token from penultimate layer
  hidden_states_2.append(output[1]) #Using representation of each CLS token

In [33]:
hidden_states = hidden_states[0].transpose(2,1)
print(hidden_states.shape)

torch.Size([100, 768, 450])


SVM on BERT Embeddings

In [108]:
from sklearn import svm

In [101]:
x = hidden_states_2[0].detach().numpy() #using classical methods on obtained embeddings from hidden states

In [109]:
svm = svm.SVC()
svm.fit(x,train_imdb['label'][:100])

ValueError: ignored

In [70]:
temp_conv_2(temp_conv(hidden_states)).shape

torch.Size([100, 1, 149])

In [72]:
pool = nn.AvgPool1d(kernel_size = 10,stride = 10) #Tried maxpooling
pool

AvgPool1d(kernel_size=(10,), stride=(10,), padding=(0,))

In [95]:
pool(temp_conv_2(temp_conv(hidden_states))).shape[2]

14

In [74]:
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
linear = nn.Linear(14,2)

In [92]:
out = sigmoid(linear(relu(pool(temp_conv(hidden_states)))))
out.shape

torch.Size([100, 30, 2])

In [93]:
out = out.detach().numpy()
np.argmax(out, axis = 1)

array([[13,  6],
       [13, 20],
       [18, 15],
       [18,  2],
       [13, 15],
       [18, 20],
       [18, 20],
       [13,  1],
       [13,  4],
       [13,  1],
       [13, 27],
       [18,  0],
       [18,  2],
       [18, 22],
       [13, 10],
       [13, 20],
       [18,  0],
       [18,  3],
       [18,  0],
       [18,  0],
       [13, 26],
       [18,  1],
       [13, 26],
       [13,  8],
       [18,  1],
       [13, 19],
       [13, 25],
       [18, 27],
       [13, 25],
       [18, 10],
       [18, 20],
       [18,  0],
       [18,  2],
       [13,  1],
       [13, 19],
       [18, 20],
       [18, 20],
       [13,  7],
       [18,  6],
       [18, 27],
       [18, 20],
       [18,  2],
       [13, 28],
       [13,  0],
       [18,  8],
       [18, 16],
       [18, 20],
       [18, 27],
       [18, 22],
       [18,  0],
       [13, 20],
       [18, 20],
       [13,  0],
       [13, 15],
       [13, 16],
       [13,  1],
       [13, 15],
       [18,  0],
       [18, 20